# Projet Knowledge Extraction - Partie A : Preprocessing et Représentation Text

**Université Paris Cité - Master 2 VMI**
**Cours :** IFLCE085 Recherche et extraction sémantique à partir de texte (Prof. Salima Benbernou)

**Équipe :**
- **Partie A (Preprocessing) : Jacques Gastebois**
- Partie B : Boutayna EL MOUJAOUID
- Partie C : Franz Dervis
- Partie D : Aya Benkabour

---

## Étape 1 : Setup et Importations
Objectif : Configurer l'environnement et importer les librairies nécessaires.

In [1]:
import sys
# Installation des dépendances de base
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install spacy

# Téléchargement du modèle spaCy anglais
!{sys.executable} -m spacy download en_core_web_sm



import os
import json
import re
import pickle
import pandas as pd
import numpy as np
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz

# Téléchargement des ressources NLTK (version mise à jour)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
nltk.download('stopwords', quiet=True)

# Chargement du modèle spaCy
nlp = spacy.load('en_core_web_sm')

# Configuration de l'affichage pandas
pd.set_option('display.max_colwidth', 100)

print("Environnement configuré avec succès.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 105.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Environnement configuré avec succès.


## Étape 2 : Chargement et Exploration des Données (SciREX)
Objectif : Télécharger (si nécessaire) et charger le dataset SciREX.

In [2]:
import urllib.request
import tarfile

DATA_DIR = "release_data"
DATA_URL = "https://github.com/allenai/SciREX/raw/master/scirex_dataset/release_data.tar.gz"
TAR_FILE = "release_data.tar.gz"

def download_and_extract_data():
    if not os.path.exists(DATA_DIR):
        print("Dossier de données non trouvé. Téléchargement en cours...")
        try:
            urllib.request.urlretrieve(DATA_URL, TAR_FILE)
            print("Téléchargement terminé. Extraction...")
            with tarfile.open(TAR_FILE, "r:gz") as tar:
                tar.extractall()
            print("Extraction terminée.")
        except Exception as e:
            print(f"Erreur lors du téléchargement/extraction : {e}")
    else:
        print("Les données sont déjà présentes.")

download_and_extract_data()

FILES = {
    "train": os.path.join(DATA_DIR, "train.jsonl"),
    "dev": os.path.join(DATA_DIR, "dev.jsonl"),
    "test": os.path.join(DATA_DIR, "test.jsonl")
}

def load_jsonl(file_path):
    """Charge un fichier JSONL dans une liste de dictionnaires."""
    data = []
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line))
    else:
        print(f"Attention : Fichier {file_path} introuvable.")
    return data

# Chargement des données
print("Chargement des données...")
train_data = load_jsonl(FILES["train"])
dev_data = load_jsonl(FILES["dev"])
test_data = load_jsonl(FILES["test"])

print(f"Nombre de documents Train : {len(train_data)}")
print(f"Nombre de documents Dev   : {len(dev_data)}")
print(f"Nombre de documents Test  : {len(test_data)}")

Les données sont déjà présentes.
Chargement des données...
Nombre de documents Train : 306
Nombre de documents Dev   : 66
Nombre de documents Test  : 66


In [3]:
# Exploration d'un document type
if train_data:
    doc_example = train_data[0]
    print("\nClés disponibles dans un document :")
    print(list(doc_example.keys()))

    print("\nExemple de contenu (champs principaux) :")
    print(f"ID: {doc_example.get('doc_id')}")
    if 'words' in doc_example:
        print(f"Début du texte (50 premiers mots) : {' '.join(doc_example['words'][:50])}...")



Clés disponibles dans un document :
['coref', 'coref_non_salient', 'doc_id', 'method_subrelations', 'n_ary_relations', 'ner', 'sections', 'sentences', 'words']

Exemple de contenu (champs principaux) :
ID: 000f90380d768a85e2316225854fc377c079b5c4
Début du texte (50 premiers mots) : Full - Resolution Residual Networks for Semantic Segmentation in Street Scenes section : Abstract Semantic image segmentation is an essential component of modern autonomous driving systems , as an accurate understanding of the surrounding scene is crucial to navigation and action planning . Current state - of - the -...


## Étape 3 : Nettoyage et Normalisation
Objectif : Nettoyer le texte (lowercase, suppression caractères spéciaux, espaces multiples).

In [4]:
def clean_text(text):
    """
    Nettoie le texte : lowercase, suppression caractères spéciaux, espaces multiples.
    
    Args:
        text (str): Texte à nettoyer
    
    Returns:
        str: Texte nettoyé
    """
    if not isinstance(text, str):
        return ""
    
    # 1. Lowercase
    text = text.lower()
    
    # 2. Suppression des caractères spéciaux (garde lettres, chiffres et espaces)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    
    # 3. Suppression des espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

print("Fonction clean_text() définie.")

Fonction clean_text() définie.


In [5]:
# Test sur l'exemple
if train_data and 'words' in doc_example:
    raw_text = ' '.join(doc_example['words'])
    cleaned_text = clean_text(raw_text)
    
    print("Texte original (200 premiers caractères) :")
    print(raw_text[:200])
    print("\nTexte nettoyé (200 premiers caractères) :")
    print(cleaned_text[:200])
    print(f"\nLongueur originale : {len(raw_text)} caractères")
    print(f"Longueur nettoyée  : {len(cleaned_text)} caractères")

Texte original (200 premiers caractères) :
Full - Resolution Residual Networks for Semantic Segmentation in Street Scenes section : Abstract Semantic image segmentation is an essential component of modern autonomous driving systems , as an acc

Texte nettoyé (200 premiers caractères) :
full resolution residual networks for semantic segmentation in street scenes section abstract semantic image segmentation is an essential component of modern autonomous driving systems as an accurate 

Longueur originale : 36348 caractères
Longueur nettoyée  : 34301 caractères


In [6]:
# Application du nettoyage sur tous les documents
print("Application du nettoyage sur tous les documents...")

for doc in train_data:
    if 'words' in doc:
        doc['cleaned_text'] = clean_text(' '.join(doc['words']))

for doc in dev_data:
    if 'words' in doc:
        doc['cleaned_text'] = clean_text(' '.join(doc['words']))

for doc in test_data:
    if 'words' in doc:
        doc['cleaned_text'] = clean_text(' '.join(doc['words']))

print(f"Nettoyage terminé pour {len(train_data)} docs train, {len(dev_data)} docs dev, {len(test_data)} docs test.")
print(f"\nExemple de texte nettoyé (doc 0) : {train_data[0]['cleaned_text'][:150]}...")

Application du nettoyage sur tous les documents...
Nettoyage terminé pour 306 docs train, 66 docs dev, 66 docs test.

Exemple de texte nettoyé (doc 0) : full resolution residual networks for semantic segmentation in street scenes section abstract semantic image segmentation is an essential component of...


## Étape 4 : Tokenization, POS Tagging et Lemmatization
Objectif : Tokeniser, identifier les parties du discours (POS) et lemmatiser les textes.

In [7]:
def tokenize_and_pos(text):
    """
    Tokenise le texte et effectue le POS tagging avec NLTK.
    
    Args:
        text (str): Texte à tokeniser
    
    Returns:
        list: Liste de tuples (token, pos_tag)
    """
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return pos_tags

def lemmatize_text(text):
    """
    Lemmatise le texte avec spaCy.
    
    Args:
        text (str): Texte à lemmatiser
    
    Returns:
        str: Texte lemmatisé
    """
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas)

print("Fonctions tokenize_and_pos() et lemmatize_text() définies.")

Fonctions tokenize_and_pos() et lemmatize_text() définies.


In [8]:
# Test sur l'exemple
if train_data and 'cleaned_text' in train_data[0]:
    sample_text = train_data[0]['cleaned_text'][:500]  # Premier 500 caractères
    
    print("Texte nettoyé (extrait) :")
    print(sample_text)
    
    print("\n--- Tokenization + POS Tagging ---")
    pos_tags = tokenize_and_pos(sample_text)
    print(f"Nombre de tokens : {len(pos_tags)}")
    print(f"Premiers 10 tokens avec POS : {pos_tags[:10]}")
    
    print("\n--- Lemmatization ---")
    lemmatized = lemmatize_text(sample_text)
    print(f"Texte lemmatisé (extrait) : {lemmatized[:200]}...")

Texte nettoyé (extrait) :
full resolution residual networks for semantic segmentation in street scenes section abstract semantic image segmentation is an essential component of modern autonomous driving systems as an accurate understanding of the surrounding scene is crucial to navigation and action planning current state of the art approaches in semantic image segmentation rely on pretrained networks that were initially developed for classifying images as a whole while these networks exhibit outstanding recognition perf

--- Tokenization + POS Tagging ---
Nombre de tokens : 70
Premiers 10 tokens avec POS : [('full', 'JJ'), ('resolution', 'NN'), ('residual', 'JJ'), ('networks', 'NNS'), ('for', 'IN'), ('semantic', 'JJ'), ('segmentation', 'NN'), ('in', 'IN'), ('street', 'NN'), ('scenes', 'NNS')]

--- Lemmatization ---
Texte lemmatisé (extrait) : full resolution residual network for semantic segmentation in street scene section abstract semantic image segmentation be an essential componen

In [9]:
# Application de la lemmatization sur TOUS les documents train
print(f"Application de la lemmatization sur {len(train_data)} documents train...")
print("Cela peut prendre quelques minutes...\n")

for i, doc in enumerate(train_data):
    if 'cleaned_text' in doc:
        doc['lemmatized_text'] = lemmatize_text(doc['cleaned_text'])
    if (i + 1) % 50 == 0:
        print(f"  Traité {i + 1}/{len(train_data)} documents...")

print(f"\nLemmatization terminée pour {len(train_data)} documents.")
print(f"Exemple de texte lemmatisé (doc 0) : {train_data[0]['lemmatized_text'][:150]}...")

Application de la lemmatization sur 306 documents train...
Cela peut prendre quelques minutes...

  Traité 50/306 documents...
  Traité 100/306 documents...
  Traité 150/306 documents...
  Traité 200/306 documents...
  Traité 250/306 documents...
  Traité 300/306 documents...

Lemmatization terminée pour 306 documents.
Exemple de texte lemmatisé (doc 0) : full resolution residual network for semantic segmentation in street scene section abstract semantic image segmentation be an essential component of m...


## Étape 5 : Représentation Vectorielle TF-IDF
Objectif : Créer une représentation vectorielle des textes avec TF-IDF sur les textes lemmatisés.

In [10]:
# Préparation des textes pour TF-IDF (utilisation des textes LEMMATISÉS)
train_texts = [doc['lemmatized_text'] for doc in train_data if 'lemmatized_text' in doc]

# Création du vectoriseur TF-IDF
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,  # Limite à 5000 features les plus importantes
    min_df=2,           # Ignore les termes qui apparaissent dans moins de 2 documents
    max_df=0.8,         # Ignore les termes qui apparaissent dans plus de 80% des documents
    ngram_range=(1, 2)  # Unigrammes et bigrammes
)

# Calcul de la matrice TF-IDF
print("Calcul de la matrice TF-IDF sur les textes lemmatisés...")
tfidf_matrix = tfidf_vectorizer.fit_transform(train_texts)

print(f"\nMatrice TF-IDF créée :")
print(f"  Forme : {tfidf_matrix.shape}")
print(f"  Nombre de documents : {tfidf_matrix.shape[0]}")
print(f"  Nombre de features : {tfidf_matrix.shape[1]}")
print(f"  Densité : {tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1]):.4f}")

Calcul de la matrice TF-IDF sur les textes lemmatisés...

Matrice TF-IDF créée :
  Forme : (306, 5000)
  Nombre de documents : 306
  Nombre de features : 5000
  Densité : 0.2372


In [11]:
# Affichage des top features pour le premier document
feature_names = tfidf_vectorizer.get_feature_names_out()
doc_0_vector = tfidf_matrix[0].toarray()[0]
top_indices = doc_0_vector.argsort()[-10:][::-1]

print("Top 10 features TF-IDF pour le document 0 :")
for idx in top_indices:
    print(f"  {feature_names[idx]}: {doc_0_vector[idx]:.4f}")

Top 10 features TF-IDF pour le document 0 :
  stream: 0.3741
  residual: 0.2400
  cityscape: 0.2283
  resolution: 0.1856
  segmentation: 0.1815
  image: 0.1728
  reference reference: 0.1653
  resnet: 0.1530
  pool: 0.1446
  boundary: 0.1343


## Étape 6 : Export et Sauvegarde des Résultats
Objectif : Sauvegarder tous les résultats pour la Partie B.

In [12]:
# Création du dossier de sortie
OUTPUT_DIR = "preprocessed_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Dossier de sortie créé : {OUTPUT_DIR}/")

Dossier de sortie créé : preprocessed_data/


### 6.1 - Export des textes prétraités (CSV)

In [13]:
# Création d'un DataFrame avec toutes les versions du texte
train_df = pd.DataFrame([
    {
        'doc_id': doc.get('doc_id', f'doc_{i}'),
        'raw_text': ' '.join(doc.get('words', [])),
        'cleaned_text': doc.get('cleaned_text', ''),
        'lemmatized_text': doc.get('lemmatized_text', '')
    }
    for i, doc in enumerate(train_data)
])

# Sauvegarde en CSV
train_df.to_csv(os.path.join(OUTPUT_DIR, 'train_preprocessed.csv'), index=False, encoding='utf-8')
print(f"✅ Fichier sauvegardé : {OUTPUT_DIR}/train_preprocessed.csv")
print(f"   Colonnes : {list(train_df.columns)}")
print(f"   Nombre de lignes : {len(train_df)}")

✅ Fichier sauvegardé : preprocessed_data/train_preprocessed.csv
   Colonnes : ['doc_id', 'raw_text', 'cleaned_text', 'lemmatized_text']
   Nombre de lignes : 306


### 6.2 - Export de la matrice TF-IDF

In [14]:
# Sauvegarde de la matrice TF-IDF (format sparse)
save_npz(os.path.join(OUTPUT_DIR, 'tfidf_matrix.npz'), tfidf_matrix)
print(f"✅ Matrice TF-IDF sauvegardée : {OUTPUT_DIR}/tfidf_matrix.npz")

# Sauvegarde du vectoriseur (pour réutilisation)
with open(os.path.join(OUTPUT_DIR, 'tfidf_vectorizer.pkl'), 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)
print(f"✅ Vectoriseur TF-IDF sauvegardé : {OUTPUT_DIR}/tfidf_vectorizer.pkl")

# Sauvegarde des noms de features
np.save(os.path.join(OUTPUT_DIR, 'tfidf_feature_names.npy'), feature_names)
print(f"✅ Noms des features sauvegardés : {OUTPUT_DIR}/tfidf_feature_names.npy")

✅ Matrice TF-IDF sauvegardée : preprocessed_data/tfidf_matrix.npz
✅ Vectoriseur TF-IDF sauvegardé : preprocessed_data/tfidf_vectorizer.pkl
✅ Noms des features sauvegardés : preprocessed_data/tfidf_feature_names.npy


### 6.3 - Export du dictionnaire de correspondance

In [15]:
# Création d'un dictionnaire de correspondance complet
correspondence_dict = {
    'metadata': {
        'n_documents': len(train_data),
        'tfidf_shape': tfidf_matrix.shape,
        'n_features': len(feature_names),
        'preprocessing_steps': [
            '1. Lowercase',
            '2. Suppression caractères spéciaux',
            '3. Normalisation espaces',
            '4. Tokenization (NLTK)',
            '5. POS Tagging (NLTK)',
            '6. Lemmatization (spaCy)',
            '7. TF-IDF sur textes lemmatisés (max_features=5000, ngram_range=(1,2))'
        ]
    },
    'documents': [
        {
            'doc_id': doc.get('doc_id', f'doc_{i}'),
            'raw_text_preview': ' '.join(doc.get('words', []))[:200],
            'cleaned_text_preview': doc.get('cleaned_text', '')[:200],
            'lemmatized_text_preview': doc.get('lemmatized_text', '')[:200],
            'tfidf_vector_index': i
        }
        for i, doc in enumerate(train_data[:10])  # Limité aux 10 premiers pour la démo
    ]
}

# Sauvegarde en JSON
with open(os.path.join(OUTPUT_DIR, 'correspondence_dict.json'), 'w', encoding='utf-8') as f:
    json.dump(correspondence_dict, f, indent=2, ensure_ascii=False)

print(f"✅ Dictionnaire de correspondance sauvegardé : {OUTPUT_DIR}/correspondence_dict.json")

✅ Dictionnaire de correspondance sauvegardé : preprocessed_data/correspondence_dict.json


### 6.4 - Résumé des fichiers exportés

In [16]:
import os

print("\n" + "="*60)
print("RÉSUMÉ DES FICHIERS EXPORTÉS")
print("="*60)

for filename in os.listdir(OUTPUT_DIR):
    filepath = os.path.join(OUTPUT_DIR, filename)
    if os.path.isfile(filepath):
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"📄 {filename:35s} ({size_mb:.2f} MB)")

print("="*60)


RÉSUMÉ DES FICHIERS EXPORTÉS
📄 tfidf_matrix.npz                    (1.97 MB)
📄 correspondence_dict.json            (0.01 MB)
📄 tfidf_feature_names.npy             (0.06 MB)
📄 tfidf_vectorizer.pkl                (0.19 MB)
📄 train_preprocessed.csv              (25.73 MB)


## Résumé Technique - Partie A

### Pipeline de Preprocessing Complet

1. **Nettoyage** :
   - Conversion en lowercase
   - Suppression des caractères spéciaux (garde uniquement lettres, chiffres, espaces)
   - Normalisation des espaces multiples

2. **Tokenization** :
   - Tokenization avec NLTK (`word_tokenize`)

3. **POS Tagging** :
   - POS tagging avec NLTK (`pos_tag`)

4. **Lemmatization** :
   - Lemmatization avec spaCy (`en_core_web_sm`)
   - **Appliqué sur TOUS les 306 documents train**

5. **Représentation TF-IDF** :
   - Vectorisation avec scikit-learn `TfidfVectorizer`
   - **Calculé sur les textes lemmatisés** (pipeline complet)
   - Paramètres : `max_features=5000`, `min_df=2`, `max_df=0.8`, `ngram_range=(1,2)`
   - Matrice résultante : 306 documents × 5000 features

---

### Fichiers Exportés pour la Partie B

| Fichier | Description | Format |
|---------|-------------|--------|
| `train_preprocessed.csv` | Textes bruts, nettoyés et lemmatisés | CSV |
| `tfidf_matrix.npz` | Matrice TF-IDF (sparse) | NumPy compressed |
| `tfidf_vectorizer.pkl` | Vectoriseur TF-IDF entraîné | Pickle |
| `tfidf_feature_names.npy` | Noms des 5000 features TF-IDF | NumPy |
| `correspondence_dict.json` | Métadonnées et correspondances | JSON |

---

### Comment Charger les Données (Partie B)

```python
import pandas as pd
import numpy as np
import pickle
from scipy.sparse import load_npz

# Charger les textes prétraités
df = pd.read_csv('preprocessed_data/train_preprocessed.csv')

# Charger la matrice TF-IDF
tfidf_matrix = load_npz('preprocessed_data/tfidf_matrix.npz')

# Charger le vectoriseur (pour transformer de nouveaux textes)
with open('preprocessed_data/tfidf_vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# Charger les noms de features
feature_names = np.load('preprocessed_data/tfidf_feature_names.npy')
```

---

### Statistiques Finales

- **Documents train** : 306 (tous lemmatisés)
- **Documents dev** : 66
- **Documents test** : 66
- **Features TF-IDF** : 5000
- **Pipeline** : Nettoyage → Tokenization → POS → Lemmatization → TF-IDF

---

**Note** : Le pipeline complet garantit que TF-IDF est calculé sur des textes entièrement prétraités (lemmatisés), ce qui améliore la qualité des représentations vectorielles.

In [19]:
# Ajouter au début du notebook
from google.colab import drive
drive.mount('/content/drive')

# Puis copier les fichiers
!cp -r preprocessed_data /content/drive/MyDrive/

KeyboardInterrupt: 